In [5]:
import subprocess
import sys

# List of required libraries
required_libraries = [
    "tkinter",         # Tkinter for GUI
    "matplotlib",      # Matplotlib for plotting graphs
    "numpy",           # NumPy for numerical operations
]

# Function to check and install libraries
def install_libraries():
    for library in required_libraries: # Loop through each library in the list
        try:
            __import__(library) # Try to import the library
        except ImportError:  # If the library is not installed, an ImportError is raised
            print(f"Library '{library}' is not installed. Installing now...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", library])  # Install the missing library using pip
# Call the function
install_libraries()

import csv
import random
import os

def generate_health_csv(file_name, num_records):  # Generate health data CSV
    headers = [
        "name", "age", "gender", "blood_pressure",
        "heart_rate", "blood_sugar", "weight", "height" ]
    genders = ["Male", "Female"] # List of possible genders
    with open(file_name, mode='w', newline='', encoding='utf-8') as csvfile: # Open the CSV file for writing
        writer = csv.DictWriter(csvfile, fieldnames=headers) # Create a CSV writer object with the specified headers
        writer.writeheader()  # Write the header row to the CSV file
        for i in range(num_records):  # Loop to generate the specified number of records
            writer.writerow({
                "name": f"Person_{i + 1}",
                "age": random.randint(18, 80),
                "gender": random.choice(genders),
                "blood_pressure": random.randint(80, 160),
                "heart_rate": random.randint(50, 120),
                "blood_sugar": random.randint(60, 220),
                "weight": round(random.uniform(40, 100), 1),
                "height": round(random.uniform(140, 200), 1)})
    print(f"CSV file '{file_name}' with {num_records} records has been generated.")
    print(f"File is stored at: {os.path.abspath(file_name)}")  # Print the absolute file path

# Generate 10 data as initial CSV
if __name__ == "__main__":
    file_name = "health_data.csv"  
    num_records = 10 
    generate_health_csv(file_name, num_records)

import tkinter as tk # Import tkinter to create GUI windows
from tkinter import messagebox # Import messagebox for displaying messages in the GUI
import matplotlib.pyplot as plt  # Import matplotlib for creating plots
import csv # Import the csv module to read and write CSV files
import time
import random
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg  #Import FigureCanvasTkAgg to display plots in Tkinter

class Person: # Define the Person class to store individual health data
    def __init__(self, name, age, gender, blood_pressure, heart_rate, blood_sugar, weight, height): # Initialize a new person
        self.name = name
        self.age = int(age)
        self.gender = gender
        self.blood_pressure = int(blood_pressure)
        self.heart_rate = int(heart_rate)
        self.blood_sugar = int(blood_sugar)
        self.weight = float(weight)
        self.height = float(height)
        latitude = random.uniform(-90, 90) # Generate a random latitude and longtitude
        longitude = random.uniform(-180, 180)
        self.location = (latitude, longitude)   # Store the location as a tuple of latitude and longitude
        self.health_history = []  # Initialize an empty list to store health history records
    def add_health_record(self): # Function to add a new health record
        self.health_history.append({
            "blood_pressure": self.blood_pressure,
            "heart_rate": self.heart_rate,
            "blood_sugar": self.blood_sugar,
            "time": time.strftime("%Y-%m-%d %H:%M:%S")}) # Add the current time as the timestamp for the record
    def check_health_status(self): # Function to check the person's health status
        abnormal_data = []  # Initialize an empty list to store any abnormal health data
        if not (90 <= self.blood_pressure <= 140):
            abnormal_data.append(f"Blood Pressure: {self.blood_pressure} (Exceeds normal range)")
        if not (60 <= self.heart_rate <= 100):
            abnormal_data.append(f"Heart Rate: {self.heart_rate} (Exceeds normal range)")
        if not (70 <= self.blood_sugar <= 180):
            abnormal_data.append(f"Blood Sugar: {self.blood_sugar} (Exceeds normal range)")
        return abnormal_data

class HealthMonitoringSystem: #Define the HealthMonitoringSystem class to manage multiple persons
    def __init__(self):
        self.persons = []
    def add_person(self, person):# Check if the person is already in the system
        if not any(p.name.lower() == person.name.lower() for p in self.persons):
            self.persons.append(person)
        else:
            print(f"{person.name} already exists in the system.")
    def check_health_and_alert(self):
        alert_persons = []
        for person in self.persons: # Loop through each person in the system
            abnormal_data = person.check_health_status() # Check the person's health status
            if abnormal_data:
                alert_persons.append((person, abnormal_data)) # Add the person and their abnormal data to the alert list
        return alert_persons
    def load_data_from_csv(self, file_name): # Function to load health data from a CSV file
        try:
            with open(file_name, newline='', encoding='utf-8') as csvfile: #Open the CSV file
                reader = csv.DictReader(csvfile) # Create a CSV reader to read the file
                for row in reader:  # Loop through each row in the CSV file
                    person = Person(row['name'], row['age'], row['gender'], row['blood_pressure'],
                                    row['heart_rate'], row['blood_sugar'], row['weight'], row['height'])
                    self.add_person(person)  # Add the person to the health monitoring system
        except FileNotFoundError as e:
            print(f"CSV file not found: {e}")

class HealthApp:
    def __init__(self, root):# Initialize the HealthApp with the root window and set up the system
        self.root = root # Store the root window reference
        self.root.title("Health Monitoring System")  # Set the title of the window
        self.hms = HealthMonitoringSystem() # Create an instance of the HealthMonitoringSystem
        self.create_widgets() # Call method to create all widgets (buttons)       
    def create_widgets(self): # Method to create the GUI widgets
        self.add_person_button = tk.Button(self.root, text="Add Person", command=self.add_person)
        self.add_person_button.pack(pady=10)# Button to add a person, calling the add_person method when clicked
        self.check_health_button = tk.Button(self.root, text="Check Health", command=self.check_health)
        self.check_health_button.pack(pady=10) # Button to check the health of a specific person
        self.show_alerts_button = tk.Button(self.root, text="Show Alerts", command=self.show_alerts)
        self.show_alerts_button.pack(pady=10) # Button to show alerts based on health status
        self.plot_trends_button = tk.Button(self.root, text="Plot Health", command=self.plot_health)
        self.plot_trends_button.pack(pady=10)# Button to plot health
        self.load_data_button = tk.Button(self.root, text="Load Data from CSV", command=self.load_data_from_csv)
        self.load_data_button.pack(pady=10)# Button to load data from a CSV file
        self.show_all_data_button = tk.Button(self.root, text="Show All Data", command=self.show_all_data)
        self.show_all_data_button.pack(pady=10) # Button to display all data of persons in the system
    def add_person(self):
        def on_submit():# Collect user input for person's details
            name = name_entry.get()
            age = age_entry.get()
            gender = gender_var.get()
            weight = random.uniform(40, 100)
            height = random.uniform(140, 200)
            blood_pressure = random.randint(80, 160)
            heart_rate = random.randint(50, 120)
            blood_sugar = random.randint(60, 220)
            person = Person(name, age, gender, blood_pressure, heart_rate, blood_sugar, weight, height)
            self.hms.add_person(person) # Add person to the HealthMonitoringSystem
            messagebox.showinfo("Success", f"Added person: {name}") # Show success message
            add_window.destroy()  # Close the "Add Person" window
        # Create a new window to input data for the new person
        add_window = tk.Toplevel(self.root)
        add_window.title("Add New Person")
        # Add label and input field for name
        tk.Label(add_window, text="Name:").pack()
        name_entry = tk.Entry(add_window)
        name_entry.pack()
        # Add label and input field for age
        tk.Label(add_window, text="Age:").pack()
        age_entry = tk.Entry(add_window)
        age_entry.pack()
        # Add label and radio buttons for gender selection
        tk.Label(add_window, text="Gender:").pack()
        gender_var = tk.StringVar(value="Male")
        tk.Radiobutton(add_window, text="Male", variable=gender_var, value="Male").pack()
        tk.Radiobutton(add_window, text="Female", variable=gender_var, value="Female").pack()
        # Submit button to submit the form data
        tk.Button(add_window, text="Submit", command=on_submit).pack(pady=10)
    def check_health(self):
        def on_submit(): # Inner function to handle health check for a person
            name = name_entry.get()   # Get the name entered by the user
            person_found = None  # Initialize a variable to store the found person
            for person in self.hms.persons:
                if person.name.lower() == name.lower():  # Search for the person by name
                    person_found = person
                    break
            if person_found: # If person is found, check their health status
                abnormal_data = person_found.check_health_status()
                full_info = f"Name: {person_found.name}\n"
                full_info += f"Age: {person_found.age}\n"
                full_info += f"Gender: {person_found.gender}\n"
                full_info += f"Blood Pressure: {person_found.blood_pressure}\n"
                full_info += f"Heart Rate: {person_found.heart_rate}\n"
                full_info += f"Blood Sugar: {person_found.blood_sugar}\n"
                full_info += f"Location: {person_found.location}\n"         
                if abnormal_data: # If abnormal data exists, display it in the message box
                    health_report = f"{full_info}Health Issues: {', '.join(abnormal_data)}"
                    messagebox.showinfo("Health Check Report", health_report)
                else:
                    messagebox.showinfo("Health Check", f"{full_info}{person_found.name} is healthy.")
            else: # Show error if the person was not found
                messagebox.showinfo("Person Not Found", "No person found with that name.")
        check_window = tk.Toplevel(self.root) # Create a new window for health check input
        check_window.title("Check Health for Specific Person")
        tk.Label(check_window, text="Enter Name:").pack() # Add label and input field for entering the name
        name_entry = tk.Entry(check_window)
        name_entry.pack()
        tk.Button(check_window, text="Submit", command=on_submit).pack(pady=10) # Submit button to perform the health check
    def show_alerts(self): 
        alert_persons = self.hms.check_health_and_alert() # Get persons with health issues
        if alert_persons:
            # Sort the alert_persons by blood pressure
            alert_persons.sort(key=lambda x: x[0].blood_pressure)  # Sorting by blood pressure in ascending order
            alerts = ""  # Initialize the alert message string
            for person, data in alert_persons:
                alerts += f"{person.name}: {', '.join(data)}\n"  # Add person and their health issues to the alert
            messagebox.showinfo("Health Alerts", alerts)  # Display the alerts
        else:
            messagebox.showinfo("No Alerts", "No health issues detected.")
    def plot_health(self):
        alert_persons = self.hms.check_health_and_alert() # Get persons with health issues
        if not alert_persons:
            messagebox.showinfo("No Alerts", "No health to plot.")  # Show message if no health issues
            return
        names = [person[0].name for person in alert_persons]  # Extract names for plotting
        blood_pressures = [person[0].blood_pressure for person in alert_persons]   # Extract blood pressure data
        heart_rates = [person[0].heart_rate for person in alert_persons]  # Extract heart rate data
        blood_sugars = [person[0].blood_sugar for person in alert_persons]   # Extract blood sugar data
        if names:
            fig, ax = plt.subplots(figsize=(10, 6))  # Create a plot figure
            ax.plot(names, blood_pressures, label="Blood Pressure") # Plot blood pressure
            ax.plot(names, heart_rates, label="Heart Rate")   # Plot heart rate
            ax.plot(names, blood_sugars, label="Blood Sugar")   # Plot blood sugar
            ax.set_xlabel("Persons")  # Set x-axis label
            ax.set_ylabel("Values") # Set y-axis label
            ax.set_title("Health data")  # Set plot title
            ax.legend()  # Show plot legend
            # Integrate the Matplotlib plot into Tkinter window
            plot_window = tk.Toplevel(self.root)  
            plot_window.title("Health Plot")
            canvas = FigureCanvasTkAgg(fig, master=plot_window) # Create canvas to embed plot
            canvas.draw() # Draw the plot
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True) # Pack the canvas into the window
        else:
            messagebox.showinfo("No Data", "No health data available to plot.")

    def load_data_from_csv(self):
        self.hms.load_data_from_csv('health_data.csv') # Load the CSV data into the system
        messagebox.showinfo("Data Loaded", "Health data loaded successfully.")

    def show_all_data(self):
        all_data = "" # Initialize the string to hold all data
        for person in self.hms.persons:
            all_data += f"Name: {person.name}, Age: {person.age}, Gender: {person.gender}, " \
                        f"Blood Pressure: {person.blood_pressure}, Heart Rate: {person.heart_rate}, " \
                        f"Blood Sugar: {person.blood_sugar}, Weight: {person.weight}, Height: {person.height}, " \
                        f"Location: {person.location}\n"  # Add each person's data to the string
        if all_data:# Display the data if available
            messagebox.showinfo("All Data", all_data)
        else:
            messagebox.showinfo("No Data", "No person data found in the system.")

# Create the Tkinter window
root = tk.Tk()
app = HealthApp(root) # Create an instance of the HealthApp class
root.mainloop()


CSV file 'health_data.csv' with 10 records has been generated.
File is stored at: /Users/suefeng/AAAAAAAAAAAAAAAAA/health_data.csv
